In [1]:
!pip install openai

In [ ]:
# Create the OpenAI client
from openai import OpenAI
from os import environ

client = OpenAI(api_key=environ['OPENAI'])

# Create the request
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
   {"role": "user", "content": "I have these notes with book titles and authors: New releases this week! The Beholders by Hester Musson, The Mystery Guest by Nita Prose. Please organize the titles and authors in a json file."}
  ],
  # Specify the response format
  response_format = {'type':'json_object'}
)

# Print the response
print(response.choices[0].message.content)

In [ ]:
message = {'role': 'user',
 'content': 'Here are some made-up addresses and company names, write them in json format. PurpleLabs Solutions, 123 Main Street, Suite 100, Anytown, USA. InnovateNow Enterprises, 789 Oak Avenue, Suite 300, Innovation City, USA. PeakPerformance Inc., 456 Elm Street, Suite 200, Dreamville, USA'}
# Use the try statement
from openai import AuthenticationError

try:
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[message])
    # Print the response
    print(response.choices[0].message.content)
# Use the except statement
except AuthenticationError as e:
    print("Please double check your authentication key and try again, the one provided is not valid.")

In [ ]:
# Import the tenacity library
from tenacity import retry,wait_random_exponential, stop_after_attempt


# Add the appropriate parameters to the decorator
@retry(wait=wait_random_exponential(min=5,max=40), stop=stop_after_attempt(4))
def get_response(model, message):
    response = client.chat.completions.create(
      model=model,
      messages=[message]
    )
    return response.choices[0].message.content
print(get_response("gpt-4o-mini", {"role": "user", "content": "List ten holiday destinations."}))

In [ ]:
measurements = [5.2, 6.3, 3.7]

messages = []
# Provide a system message and user messages to send the batch
messages.append({'role':'system','content':'Generate conversion from kilometers to miles of the given measurement requests in batches and present the results as a table of both the original and converted measurements.'})
# Append measurements to the message
[messages.append({'role':'user','content': f'{i}'}) for i in measurements]

response = get_response(messages)
print(response)

In [ ]:
import tiktoken
input_message = {"role": "user", "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"}

# Use tiktoken to create the encoding for your model
encoding = tiktoken.encoding_for_model('gpt-4o-mini')
# Check for the number of tokens
num_tokens = len(encoding.encode(input_message['content']))

# Run the chat completions function and print the response
if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-4o-mini", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

In [ ]:
message_listing = [{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Step into this beautiful two-story, single-family home located in Springfield, USA, priced at $350,000. This charming property features 4 bedrooms, 2.5 bathrooms, a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a large backyard perfect for family gatherings. The master bedroom includes an en-suite bathroom and a walk-in closet. Enjoy the convenience of an attached two-car garage and a recently updated HVAC system. Located near top-rated schools, parks, and shopping centers, this home is ideal for families looking for comfort and convenience.'}]

function_definition = [{'type': 'function',
  'function': {'name': 'real_estate_info',
   'description': 'Get the information about homes for sale from the body of the input text',
   'parameters': {'type': 'object',
    'properties': {'home type': {'type': 'string', 'description': 'Home type'},
     'location': {'type': 'string', 'description': 'Location'},
     'price': {'type': 'integer', 'description': 'Price'},
     'bedrooms': {'type': 'integer', 'description': 'Number of bedrooms'}}}}}]


response= client.chat.completions.create(
    model="gpt-4o-mini",
    # Add the message
    messages=message_listing,
    # Add your function definition
    tools = function_definition
)

# Print the response
print(response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:
# Define the function parameter type
function_definition[0]['function']['parameters']['type'] = 'object'

# Define the function properties
function_definition[0]['function']['parameters']['properties'] = {'title':{'type':'string','description':'Title of the paper'},'year':{'type':'string','description':'Year of the publication'}}

response = get_response(messages, function_definition)
print(response)

In [ ]:
response = get_response(messages, function_definition)

# Define the function to extract the data dictionary
def extract_dictionary(response):
  return response.choices[0].message.tool_calls[0].function.arguments

# Print the data dictionary
print(extract_dictionary(response))

In [ ]:
response= client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    # Add the function definition
    tools=function_definition,
    # Specify the function to be called for the response
    tool_choice={'type':'function','function':{'name':'extract_review_info'}}
)

# Print the response
print(response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:

# Define the function to pass to tools
function_definition = [{"type": 'function',
                        'function' : {"name": 'get_airport_info',
                                'description': 'This function returns information on a matching airport code',
                                'parameters': {"type": 'object', 'properties': {"airport_code": {'type':'string','description':'Returns the airport information given the aiport code'}, } }, 
                                "result": {'type':'string'}}}]

response = get_response(function_definition)
print(response)

In [ ]:

# Call the Chat Completions endpoint 
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {'role':'system','content':'You are in the aviation space now and you need to extract the corresponding airport code based on the user input.'},
    {"role": 'user', "content": "I'm planning to land a plane in JFK airport in New York and would like to have the corresponding information."}],
  tools=function_definition)

print(response)

In [ ]:
if response.choices[0].finish_reason=='tool_calls':
  function_call = response.choices[0].message.tool_calls[0].function
  # Check function name
  if function_call.name == 'get_airport_info':
    # Extract airport code
    code = json.loads(function_call.arguments)['airport code']
    airport_info = get_airport_info(code)
    print(airport_info)
  else:
    print("Apologies, I couldn't find any airport.")
else: 
  print("I am sorry, but I could not understand your request.")

In [ ]:
message = "Can you show some example sentences in the past tense in French?"

# Use the moderation API
moderation_response = (client.moderations.create(input=message))

# Print the response
print(moderation_response.results[0].categories.violence)

In [ ]:
import uuid

# Generate a unique ID
unique_id = str(uuid.uuid4())

response = client.chat.completions.create(  
  model="gpt-4o-mini", 
  messages=messages,
# Pass a user identification key
  user = unique_id
)

print(response.choices[0].message.content)

### Embeddings OpenAI API

In [ ]:
# Create an OpenAI client
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

# Create a request to obtain embeddings
response = client.embeddings.create(
    model = 'text-embedding-3-small',
    input = "Something goes here"
)

# Convert the response into a dictionary
response_dict = response.model_dump()
print(response_dict)
print(response_dict['data'][0]['embedding'])

In [ ]:
products = [{'title': 'Smartphone X1', 'short_description': 'The latest flagship smartphone with AI-powered features and 5G connectivity.', 'price': 799.99, 'category': 'Electronics', 'features': ['6.5-inch AMOLED display', 'Quad-camera system with 48MP main sensor', 'Face recognition and fingerprint sensor', 'Fast wireless charging']}]

# Extract a list of product short descriptions from products
product_descriptions = [product['short_description'] for product in products]

# Create embeddings for each product description
response = client.embeddings.create(
    model = 'text-embedding-3-small',
    input = product_descriptions
)
response_dict = response.model_dump()

# Extract the embeddings from response_dict and store in products
for i, product in enumerate(products):
    product['embedding'] = response_dict['data'][0]['embedding']
    
print(products[0].items())

In [ ]:
# Create reviews and embeddings lists using list comprehensions
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

categories = [product['category'] for product in products]
embeddings = [product['embedding'] for product in products]

# Reduce the number of embeddings dimensions to two using t-SNE
tsne = TSNE(n_components=2, perplexity=5)
embeddings_2d = tsne.fit_transform(np.array(embeddings))
print(embeddings_2d.shape)
# Create a scatter plot from embeddings_2d
plt.scatter(embeddings_2d[:,0], embeddings_2d[:,1])

for i, category in enumerate(categories):
    plt.annotate(category, (embeddings_2d[i, 0], embeddings_2d[i, 1]))

plt.show()

In [ ]:
# Define a create_embeddings function
def create_embeddings(texts):
  response = client.embeddings.create(
    model="text-embedding-3-small",
    input=texts
  )
  response_dict = response.model_dump()
  
  return [data['embedding'] for data in response_dict['data']]

short_description = 'The latest flagship smartphone with AI-powered features and 5G connectivity.'
list_of_descriptions = ['Charge your devices conveniently with this sleek wireless charging dock.',
 'Elevate your skincare routine with this luxurious skincare set.']

# Embed short_description and print
print(create_embeddings(short_description)[0])

# Embed list_of_descriptions and print
print(create_embeddings(list_of_descriptions))

In [ ]:
# Embed the search text
from scipy.spatial import distance
search_text = "soap"
search_embedding = create_embeddings(search_text)[0]

distances = []
for product in products:
  # Compute the cosine distance for each product description
  dist = distance.cosine(search_embedding, product['embedding'])
  distances.append(dist)

# Find and print the most similar product short_description    
min_dist_ind = np.argmin(distances)
print(products[min_dist_ind]['short_description'])

In [ ]:
# Define a function to combine the relevant features into a single string
def create_product_text(product):
  return f"""Title: {product['title']}
Description: {product['short_description']}
Category: {product['category']}
Features: {'; '.join(product['features'])}"""

# Combine the features for each product
product_texts = [create_product_text(product) for product in products]

# Create the embeddings from product_texts
product_embeddings = create_embeddings(product_texts)

In [ ]:
def find_n_closest(query_vector, embeddings, n=3):
  distances = []
  for index, embedding in enumerate(embeddings):
    # Calculate the cosine distance between the query vector and embedding
    dist = distance.cosine(query_vector,embedding)
    # Append the distance and index to distances
    distances.append({"distance": dist, "index": index})
  # Sort distances by the distance key
  distances_sorted = sorted(distances,key=lambda x:x['distance'])
  # Return the first n elements in distances_sorted
  return distances_sorted[:n]